In [1]:
import pandas as pd
import numpy as np
import glob
import os
import math
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

Import cricsheet ball-by-ball data for Men's test matches and store in dataframes (one for each match/csv file). 

In [2]:
filepath = '/Users/lkc207/Documents/Cricket/Match_prediction/cricsheet_data/test_csv_male/'
all_files = glob.glob(filepath + "/*.csv")

df_list=[]

for filename in all_files:
    df = pd.read_csv(filename, skiprows=0,  names = ["Ball", "Innings", "Over", "Batting_team","Batter","Non-striker","Bowler","Runs","Extras","How_out","Batter_out"])
    file_path, file_extension = os.path.splitext(filename)
    match_id = file_path.split('/')[-1]
    df["Match_id"] = match_id  ### add the match_id extracted from filename as a column
    df["Start_date"] = df.loc[(df['Innings']=='date') & (df['Match_id']==match_id)]['Over'].iloc[0]
    df_list.append(df)

Concatenate list of dataframes into one df and reset index.

In [3]:
df_all = pd.concat(df_list).reset_index(drop=True)
df_all

,Ball,Innings,Over,Batting_team,Batter,Non-striker,Bowler,Runs,Extras,How_out,Batter_out,Match_id,Start_date
0,version,1.3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522245,2011/08/04
1,info,team,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522245,2011/08/04
2,info,team,Bangladesh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522245,2011/08/04
3,info,gender,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522245,2011/08/04
4,info,season,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522245,2011/08/04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249238,ball,3,30.3,Sri Lanka,TT Samaraweera,HAPW Jayawardene,Umar Gul,0.0,1.0,NaN,NaN,388993,2009/02/21
1249239,ball,3,30.4,Sri Lanka,TT Samaraweera,HAPW Jayawardene,Umar Gul,4.0,0.0,NaN,NaN,388993,2009/02/21
1249240,ball,3,30.5,Sri Lanka,TT Samaraweera,HAPW Jayawardene,Umar Gul,1.0,0.0,NaN,NaN,388993,2009/02/21
1249241,ball,3,30.6,Sri Lanka,HAPW Jayawardene,TT Samaraweera,Umar Gul,0.0,0.0,NaN,NaN,388993,2009/02/21


Create dictionary of venues and corresponding country -- to be able to identify home team
The country of each venue is entered into the dictionary manually.

In [4]:
#### create dictionary of venues and corresponding country -- to be able to identify home team?
venues = df_all[df_all['Innings']=='venue']['Over'].unique()
venues_dict = dict.fromkeys(venues)

### set country of each venue manually.

venues_dict['Bellerive Oval'] = 'Australia'
venues_dict['Manuka Oval'] = 'Australia'
venues_dict['Adelaide Oval'] = 'Australia'
venues_dict['Sydney Cricket Ground'] = 'Australia'
venues_dict['Melbourne Cricket Ground'] = 'Australia'
venues_dict['Brisbane Cricket Ground, Woolloongabba'] = 'Australia'
venues_dict['Western Australia Cricket Association Ground'] = 'Australia'
venues_dict['Perth Stadium'] = 'Australia'
venues_dict['W.A.C.A. Ground'] = 'Australia'

venues_dict['Chittagong Divisional Stadium'] = 'Bangladesh'
venues_dict['Zohur Ahmed Chowdhury Stadium'] = 'Bangladesh'
venues_dict['Zahur Ahmed Chowdhury Stadium'] = 'Bangladesh'
venues_dict['Shere Bangla National Stadium'] = 'Bangladesh'
venues_dict['Sheikh Abu Naser Stadium'] = 'Bangladesh'
venues_dict['Shere Bangla National Stadium, Mirpur'] = 'Bangladesh'
venues_dict['Shaheed Chandu Stadium'] = 'Bangladesh'
venues_dict['Khan Shaheb Osman Ali Stadium'] = 'Bangladesh'
venues_dict['Sylhet International Cricket Stadium'] = 'Bangladesh'

venues_dict['Old Trafford'] = 'England'
venues_dict['Headingley'] = 'England'
venues_dict['Edgbaston'] = 'England'
venues_dict['Lord\'s'] = 'England'
venues_dict['Trent Bridge'] = 'England'
venues_dict['The Rose Bowl'] = 'England'
venues_dict['Riverside Ground'] = 'England'
venues_dict['Kennington Oval'] = 'England'
venues_dict['Sophia Gardens'] = 'England'

venues_dict['Eden Gardens'] = 'India'
venues_dict['Wankhede Stadium'] = 'India'
venues_dict['M Chinnaswamy Stadium'] = 'India'
venues_dict['M.Chinnaswamy Stadium'] = 'India'
venues_dict['Green Park'] = 'India'
venues_dict['Brabourne Stadium'] = 'India'
venues_dict['Sardar Patel Stadium, Motera'] = 'India'
venues_dict['Sardar Patel Stadium'] = 'India'
venues_dict['Punjab Cricket Association Stadium, Mohali'] = 'India'
venues_dict['Punjab Cricket Association IS Bindra Stadium, Mohali'] = 'India'
venues_dict['MA Chidambaram Stadium, Chepauk'] = 'India'
venues_dict['Feroz Shah Kotla'] = 'India'
venues_dict['Arun Jaitley Stadium'] = 'India'
venues_dict['Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium'] = 'India'
venues_dict['Saurashtra Cricket Association Stadium'] = 'India'
venues_dict['Rajiv Gandhi International Stadium, Uppal'] = 'India'
venues_dict['JSCA International Stadium Complex'] = 'India'
venues_dict['Rajiv Gandhi International Cricket Stadium, Dehradun'] = 'India'
venues_dict['Holkar Cricket Stadium'] = 'India'
venues_dict['Vidarbha C.A. Ground'] = 'India'
venues_dict['Maharashtra Cricket Association Stadium'] = 'India'
venues_dict['Himachal Pradesh Cricket Association Stadium'] = 'India'
venues_dict['Vidarbha Cricket Association Stadium, Jamtha'] = 'India'
venues_dict['Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium'] = 'India'

venues_dict['The Village, Malahide'] = 'Ireland'

venues_dict['University Oval'] = 'New Zealand'
venues_dict['Hagley Oval'] = 'New Zealand'
venues_dict['Eden Park'] = 'New Zealand'
venues_dict['Bay Oval'] = 'New Zealand'
venues_dict['Seddon Park'] = 'New Zealand'
venues_dict['McLean Park'] = 'New Zealand'
venues_dict['Basin Reserve'] = 'New Zealand'
venues_dict['AMI Stadium'] = 'New Zealand'

venues_dict['Rawalpindi Cricket Stadium'] = 'Pakistan'
venues_dict['Gaddafi Stadium'] = 'Pakistan'
venues_dict['Iqbal Stadium'] = 'Pakistan'
venues_dict['Multan Cricket Stadium'] = 'Pakistan'
venues_dict['National Stadium'] = 'Pakistan'

venues_dict['Newlands'] = 'South Africa'
venues_dict['Kingsmead'] = 'South Africa'
venues_dict['St George\'s Park'] = 'South Africa'
venues_dict['Mangaung Oval'] = 'South Africa'
venues_dict['OUTsurance Oval'] = 'South Africa'
venues_dict['SuperSport Park'] = 'South Africa'
venues_dict['The Wanderers Stadium'] = 'South Africa'
venues_dict['New Wanderers Stadium'] = 'South Africa'
venues_dict['Senwes Park'] = 'South Africa'

venues_dict['R.Premadasa Stadium, Khettarama'] = 'Sri Lanka'
venues_dict['R.Premadasa Stadium'] = 'Sri Lanka'
venues_dict['R Premadasa Stadium'] = 'Sri Lanka'
venues_dict['Pallekele International Cricket Stadium'] = 'Sri Lanka'
venues_dict['Galle International Stadium'] = 'Sri Lanka'
venues_dict['P Sara Oval'] = 'Sri Lanka'
venues_dict['Sinhalese Sports Club Ground'] = 'Sri Lanka'
venues_dict['P Saravanamuttu Stadium'] = 'Sri Lanka'
venues_dict['Asgiriya Stadium'] = 'Sri Lanka'

venues_dict['Sheikh Zayed Stadium'] = 'UAE'
venues_dict['Dubai International Cricket Stadium'] = 'UAE'
venues_dict['Sharjah Cricket Stadium'] = 'UAE'

venues_dict['Queen\'s Park Oval, Port of Spain'] = 'West Indies'
venues_dict['Sir Vivian Richards Stadium, North Sound'] = 'West Indies'
venues_dict['Daren Sammy National Cricket Stadium, St Lucia'] = 'West Indies'
venues_dict['Daren Sammy National Cricket Stadium, Gros Islet'] = 'West Indies'
venues_dict['Darren Sammy National Cricket Stadium, Gros Islet'] = 'West Indies'
venues_dict['Beausejour Stadium, Gros Islet'] = 'West Indies'
venues_dict['Kensington Oval, Bridgetown'] = 'West Indies'
venues_dict['Kensington Oval'] = 'West Indies'
venues_dict['Kensington Oval, Barbados'] = 'West Indies'
venues_dict['Sabina Park, Jamaica'] = 'West Indies'
venues_dict['Sabina Park, Kingston'] = 'West Indies'
venues_dict['Arnos Vale Ground, Kingstown'] = 'West Indies'
venues_dict['Warner Park, Basseterre'] = 'West Indies'
venues_dict['Antigua Recreation Ground, St John\'s'] = 'West Indies'
venues_dict['Antigua Recreation Ground'] = 'West Indies'
venues_dict['Windsor Park, Roseau'] = 'West Indies'
venues_dict['Providence Stadium'] = 'West Indies'
venues_dict['National Cricket Stadium, St George\'s'] = 'West Indies'
venues_dict['Sir Vivian Richards Stadium, Antigua'] = 'West Indies'

venues_dict['Harare Sports Club'] = 'Zimbabwe'
venues_dict['Queens Sports Club'] = 'Zimbabwe'

#### Check all venues have been assigned their country
for key in venues_dict.keys():
    if not venues_dict[key]:
        print(key,' , ',venues_dict[key])

Remove test matches with unusual features that may impact the modelling

In [5]:
#### Match_id 225258 was awarded to England (Pakistan ball tamepring?) Remove this from records for now
df_all = df_all[df_all['Match_id'] != '225258']

In [6]:
#### Match_id 1183534 involves penalty runs. Remove this from records for now
df_all = df_all[df_all['Match_id'] != '1183534']

In [7]:
#### Match_id 1140386 involves penalty runs. Remove this from records for now
df_all = df_all[df_all['Match_id'] != '1140386']

In [8]:
#### Match_id 1030217 involves penalty runs. Remove this from records for now
df_all = df_all[df_all['Match_id'] != '1030217']

If there was no 4th innings in the match and the match was not won by an innings then assume that the match was weather affected and so remove these records for now as this may affect models.

In [9]:
df_temp = df_all[df_all['Innings'].isin(['1','2','3','4'])][['Match_id','Innings']]
df_temp = df_temp.astype({'Innings': int})
df_temp.info()
df_max_innings = df_temp.groupby(['Match_id'])[['Innings']].max().reset_index()
no4th_matchid = df_max_innings[df_max_innings['Innings']<4]['Match_id'].tolist()
#print(no4th_matchid, len(no4th_matchid))
winbyinn_match_id = df_all[(df_all['Match_id'].isin(no4th_matchid)) & (df_all['Innings']=='winner_innings')  & \
       (df_all['Over']=='1')]['Match_id'].tolist()
#print(winbyinn_match_id , len(winbyinn_match_id ))
cutshort_match_id = list(set(no4th_matchid) - set(winbyinn_match_id))
#print(cutshort_match_id , len(cutshort_match_id ))

df_all = df_all[~df_all['Match_id'].isin(cutshort_match_id)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226870 entries, 22 to 1249242
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Match_id  1226870 non-null  object
 1   Innings   1226870 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 28.1+ MB


Count number of runs scored, wickets taken and overs bowled in each innings and store in columns in df

In [10]:
df_temp = df_all.groupby(['Match_id','Innings'])[['Runs','Extras']].sum().reset_index()
df_temp['Total_runs']=df_temp['Runs']+df_temp['Extras']
df_temp.drop(df_temp[df_temp['Total_runs']==0].index, inplace=True)


df_temp2 = df_all.groupby(['Match_id','Innings'])['How_out'].count().reset_index()
df_temp2.rename(columns={"How_out": "Wickets"},inplace=True)
df_temp2 = df_temp2[df_temp2['Innings'].isin(['1','2','3','4'])]

df_new_a = pd.DataFrame(columns = ['Match_id', 'Innings1_runs', 'Innings1_wickets','Innings2_runs', 'Innings2_wickets',\
'Innings3_runs', 'Innings3_wickets','Innings4_runs', 'Innings4_wickets','Innings1_overs','Innings2_overs',\
                                   'Innings3_overs','Innings4_overs'])

df_new_a['Match_id'] = df_temp2['Match_id'].unique()

df_new_a['Innings1_runs'] = df_temp.pivot(index='Match_id',columns=['Innings'])['Total_runs']['1'].tolist()
df_new_a['Innings2_runs'] = df_temp.pivot(index='Match_id',columns=['Innings'])['Total_runs']['2'].tolist()
df_new_a['Innings3_runs'] = df_temp.pivot(index='Match_id',columns=['Innings'])['Total_runs']['3'].tolist()
df_new_a['Innings4_runs'] = df_temp.pivot(index='Match_id',columns=['Innings'])['Total_runs']['4'].tolist()

df_new_a['Innings1_wickets'] = df_temp2.pivot(index='Match_id',columns=['Innings'])['Wickets']['1'].tolist()
df_new_a['Innings2_wickets'] = df_temp2.pivot(index='Match_id',columns=['Innings'])['Wickets']['2'].tolist()
df_new_a['Innings3_wickets'] = df_temp2.pivot(index='Match_id',columns=['Innings'])['Wickets']['3'].tolist()
df_new_a['Innings4_wickets'] = df_temp2.pivot(index='Match_id',columns=['Innings'])['Wickets']['4'].tolist()

df_temp4 = df_all[df_all['Innings'].isin(['1','2','3','4'])]
df_temp4 = df_temp4.astype({'Over': float})
df_temp4 = df_temp4.groupby(['Match_id','Innings'])[['Over']].max().reset_index()

df_new_a['Innings1_overs'] = df_temp4.pivot(index='Match_id',columns=['Innings'])['Over']['1'].tolist()
df_new_a['Innings2_overs'] = df_temp4.pivot(index='Match_id',columns=['Innings'])['Over']['2'].tolist()
df_new_a['Innings3_overs'] = df_temp4.pivot(index='Match_id',columns=['Innings'])['Over']['3'].tolist()
df_new_a['Innings4_overs'] = df_temp4.pivot(index='Match_id',columns=['Innings'])['Over']['4'].tolist()

df_new_a

,Match_id,Innings1_runs,Innings1_wickets,Innings2_runs,Innings2_wickets,Innings3_runs,Innings3_wickets,Innings4_runs,Innings4_wickets,Innings1_overs,Innings2_overs,Innings3_overs,Innings4_overs
0,1000851,242.0,10.0,244.0,10.0,540.0,8.0,361.0,10.0,63.4,70.3,160.1,119.1
1,1000853,85.0,10.0,326.0,10.0,161.0,10.0,NaN,NaN,32.5,100.5,60.1,NaN
2,1000855,259.0,9.0,383.0,10.0,250.0,10.0,127.0,3.0,75.6,121.1,85.2,40.5
3,1000881,429.0,10.0,142.0,10.0,202.0,5.0,450.0,10.0,130.1,54.6,38.6,144.6
4,1000883,443.0,9.0,624.0,8.0,163.0,10.0,NaN,NaN,126.3,141.6,53.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,936149,392.0,10.0,110.0,10.0,224.0,7.0,224.0,10.0,115.6,42.6,51.5,61.6
562,936151,426.0,10.0,131.0,10.0,177.0,10.0,NaN,NaN,124.1,45.4,42.3,NaN
563,995451,117.0,10.0,203.0,10.0,353.0,10.0,161.0,10.0,34.2,79.2,93.4,88.3
564,995453,281.0,10.0,106.0,10.0,237.0,10.0,183.0,10.0,73.1,33.2,59.3,50.1


Extract and store the following information: Match_id, Home_team (team name), Away_team (team name), Start_date, Toss_winner (team name), Toss_decision (field or bat), Bat_first (team name), Field_first (team name), Outcome (team name or draw), By_innings (boolean value), Margin, Home_players (list), Away_players (list). Most variables should be self explantory.

#### I think this can be done in a more efficient way than with a loop but haven't figured it out yet. This will do for now as once the data is prepared we don't have to repeatedly run this stage.

In [11]:
df_new = pd.DataFrame(columns = ['Match_id', 'Home_team', 'Away_team', 'Start_date','Toss_winner',\
                                     'Toss_decision','Bat_first','Field_first','Outcome','By_innings','Margin',\
                                    'Home_players','Away_players'])

for counter, matchid in enumerate(df_all['Match_id'].unique()):

    

    tosswinner = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='toss_winner')]['Over'].iloc[0]
    tossdecision = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='toss_decision')]['Over'].iloc[0]

    #hometeam = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='team')]['Over'].iloc[0]
    #awayteam = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='team')]['Over'].iloc[1]
    
    venue = df_all[(df_all['Match_id']==matchid) & (df_all['Innings']=='venue')]['Over'].values[0]
    team1 = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='team')]['Over'].iloc[0]
    team2 = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='team')]['Over'].iloc[1]

    if team1 == venues_dict[venue]:
        hometeam = team1
        awayteam = team2
    elif team2 == venues_dict[venue]:
        hometeam = team2
        awayteam = team1
    else:
        hometeam = 'neutral'
        awayteam = 'neutral'
        print('CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE:',venues_dict[venue], team1, team2)

    #print(venue, venues_dict[venue], hometeam, awayteam)    
        
    if tossdecision=='field':
        field_first = tosswinner
        if field_first == hometeam:
            bat_first = awayteam
        else:
            bat_first = hometeam
    else:
        bat_first = tosswinner
        if bat_first == hometeam:
            field_first = awayteam
        else:
            field_first = hometeam

    startdate = df_all.loc[(df_all['Match_id']==matchid)]['Start_date'].iloc[0]
    print(counter, matchid)
    outcome = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner') | \
                        (df_all['Match_id']==matchid) & (df_all['Innings']=='outcome') ]['Over'].iloc[0]
    if outcome!='draw':
        #print('here',df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner_innings')])
        if not df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner_innings')].empty:
            by_innings = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner_innings')]\
                      ['Over'].iloc[0]
        else:
            by_innings = 0
                                  
        margin = [df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner_runs') | \
                             (df_all['Match_id']==matchid) & (df_all['Innings']=='winner_wickets')]['Over'].iloc[0], \
                  df_all.loc[(df_all['Match_id']==matchid) & (df_all['Innings']=='winner_runs') | \
                            (df_all['Match_id']==matchid) & (df_all['Innings']=='winner_wickets')]['Innings'].iloc[0]]
    else:
        margin = None
        by_innings = None
    
    away_batters_in_match = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Batting_team']==awayteam) ]['Batter'].dropna().unique().tolist()
    away_bowlers_in_match = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Batting_team']==hometeam)  ]['Bowler'].dropna().unique().tolist()
    away_players_in_match = list(set(away_batters_in_match+away_bowlers_in_match))

    home_batters_in_match = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Batting_team']==hometeam) ]['Batter'].dropna().unique().tolist()
    home_bowlers_in_match = df_all.loc[(df_all['Match_id']==matchid) & (df_all['Batting_team']==awayteam)  ]['Bowler'].dropna().unique().tolist()
    home_players_in_match = list(set(home_batters_in_match+home_bowlers_in_match))
    
    df_new = df_new.append({'Match_id': matchid, 'Home_team': hometeam, 'Away_team': awayteam, 'Start_date': startdate,\
                            'Toss_winner': tosswinner, 'Toss_decision':tossdecision,'Field_first':field_first,\
                            'Bat_first':bat_first,'Outcome': outcome,'By_innings': by_innings,\
                           'Margin': margin,'Home_players': home_players_in_match,'Away_players': away_players_in_match\
                            }, ignore_index=True)

0 522245
1 298794
2 913613
3 1144165
4 456671
5 352664
6 676525
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: UAE England Pakistan
7 531629
8 1144993
9 463147
10 64118
11 1233962
12 64124
13 582193
14 914237
15 582192
16 345669
17 1192873
18 463146
19 282691
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: UAE England Pakistan
20 531628
21 401072
22 1024041
23 1144164
24 1022593
25 569245
26 760781
27 903603
28 219062
29 350474
30 573009
31 760783
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: UAE Pakistan Sri Lanka
32 657649
33 531986
34 1024043
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: England Australia Pakistan
35 426395
36 426424
37 423789
38 592397
39 361050
40 361051
41 300444
42 1031659
43 1062573
44 282692
45 892509
46 401071
47 676527
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: England Australia Pakistan
48 426394
49 1130745
50 531987
51 1225247
52 573008
53 1029827
54 428752
55 250667
56 602473
57 913615
58 1152846
59 221840
CHECK THIS FIXTURE WAS PLAYED

520 800461
521 573012
522 1213062
523 1225249
524 464988
525 1144155
526 456669
527 1031441
528 380712
529 892513
530 1059702
531 319133
532 423786
533 64128
534 1075983
535 592401
536 345670
537 1030213
538 291353
539 592398
540 1188629
541 534205
542 423779
543 423778
544 1188628
545 592399
546 345671
547 291352
548 592400
549 1075982
550 319132
551 1059703
552 426402
553 426416
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: UAE England Pakistan
554 531630
555 380713
556 1000851
557 995453
558 562446
559 1144154
560 1119535
561 521225
562 1225248
563 1187686
CHECK THIS FIXTURE WAS PLAYED AT A NEUTRAL VENUE: UAE Pakistan Sri Lanka
564 657647
565 573013


Merge df_new and df_new_a dataframes using common, unique key of 'Match_id'

In [12]:
df_new = df_new.merge(df_new_a, how='inner', on='Match_id')

Make sure Match_id is an integer

In [13]:
df_new = df_new.astype({'Match_id': int})

Count how many matches have 23 or more players listed. These will be removed from the set for modelling purposes as it is not a usual occurrence. Concussion subs are one possible reason for this.

In [14]:
count_over = df_new[(df_new['Home_players'].str.len() + df_new['Away_players'].str.len()) >=23]['Match_id'].count()
print(f'{count_over} out of {len(df_new)}')

6 out of 566


Split each of the 'Home_players' and 'Away_players' lists so each player has their own column. Then remove the gamew which have more than 11 home players and/or more than 11 away players. This requires knowing the longest list length in each of 'Home_players' and 'Away_players'. 

#### At the moment this is done manually, but could be automated in future

In [15]:
df_new[['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11','p12']] \
= pd.DataFrame(df_new['Home_players'].tolist(), index= df_new.index)

#### then remove games which have a 12th home player:
df_new.drop(df_new[df_new['p12'].notna()].index, inplace=True)
df_new.drop(columns=['p12'],inplace=True)

df_new[['p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22','p23','p24']] \
= pd.DataFrame(df_new['Away_players'].tolist(), index= df_new.index)

#### then remove games which have a 12th and 13th away player:
df_new.drop(df_new[df_new['p23'].notna()].index, inplace=True)
df_new.drop(columns=['p23','p24'],inplace=True)

Reset the index

In [16]:
df_new.reset_index(inplace=True)

Because of the way the data has been extracted from the cricsheet files, if a player didn't bat or bowl in the match then they won't be listed. Find which cases are missing and manually impute the data using cricinfo. First, split the strings in the list:

#### Games played at neutral venues don't have the players listed, again because if the way the data is extracted. Remove these game for now; work on fix in future...

In [17]:
df_new = df_new[~df_new['p1'].isnull()]

Print list of match ids to impute. This will help find the relevant info on cricinfo

In [18]:
matches_to_impute = df_new[(df_new['Home_players'].str.len()<11) | (df_new['Away_players'].str.len()<11)]['Match_id'].values
print(matches_to_impute)

[ 463146  892509   64123  298803  534225  249193 1109602  474463  514034
  291338  258460  518951 1183531  210366  293480 1119550 1187008 1233957
  892517  892515]


Manually enter the missing players for these matches

In [19]:
### 463146
df_new.loc[df_new['Match_id']==463146,'p10'] = 'AG Prince'
df_new.loc[df_new['Match_id']==463146,'p11'] = 'MV Boucher'

### 892509
df_new.loc[df_new['Match_id']==892509,'p11'] = 'PM Nevill'

### 64123
df_new.loc[df_new['Match_id']==64123,'p10'] = 'AG Prince'
df_new.loc[df_new['Match_id']==64123,'p11'] = 'MV Boucher'

### 298803
df_new.loc[df_new['Match_id']==298803,'p11'] = 'MV Boucher'

### 534225
df_new.loc[df_new['Match_id']==534225,'p21'] = 'AB de Villiers'
df_new.loc[df_new['Match_id']==534225,'p22'] = 'JA Rudolph'

### 249193
df_new.loc[df_new['Match_id']==249193,'p11'] = 'HAPW Jayawardene'

### 1109602
df_new.loc[df_new['Match_id']==1109602,'p11'] = 'DAS Gunaratne'

### 474463
df_new.loc[df_new['Match_id']==474463,'p11'] = 'MJ Prior'

### 514034
df_new.loc[df_new['Match_id']==514034,'p11'] = 'MV Boucher'

### 291338
df_new.loc[df_new['Match_id']==291338,'p11'] = 'AC Gilchrist'

### 258460
df_new.loc[df_new['Match_id']==258460,'p22'] = 'RR Sarwan'

### 518951
df_new.loc[df_new['Match_id']==518951,'p11'] = 'BJ Haddin'

### 1183531
df_new.loc[df_new['Match_id']==1183531,'p10'] = 'TM Head'
df_new.loc[df_new['Match_id']==1183531,'p11'] = 'TD Paine'

### 210366
df_new.loc[df_new['Match_id']==210366,'p11'] = 'GO Jones'

### 293480
df_new.loc[df_new['Match_id']==293480,'p11'] = 'HAPW Jayawardene'

### 1119550
df_new.loc[df_new['Match_id']==1119550,'p11'] = 'AU Rashid'

### 1187008
df_new.loc[df_new['Match_id']==1187008,'p11'] = 'WP Saha'

### 1233957
df_new.loc[df_new['Match_id']==1233957,'p22'] = 'SO Dowrich'

### 892517
df_new.loc[df_new['Match_id']==892517,'p11'] = 'PM Nevill'

### 892515
df_new.loc[df_new['Match_id']==892515,'p11'] = 'PM Nevill'

Check if all missing players have been filled out

In [20]:
sw = 0
for ap in ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11',\
    'p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22']:
     if df_new[ap].isnull().sum() != 0:
        print(f'{ap} has missing players')
        sw = 1
if sw==0:
    print(f'No missing players')

No missing players


Drop Home_players and Away_players columns now as these are incomplete and can be remade later from the individual player columns if necessary

In [24]:
df_new.drop(columns=['Home_players','Away_players'],inplace=True)

Save the manipulated data up to this point 

In [26]:
df_new.to_csv('test_match_summary.csv',index=False)

Read in the summary data if starting from this point

In [27]:
df_new = pd.read_csv('test_match_summary.csv')

Import data for ICC test batting and bowling rankings in order to add player ratings to dataframe

In [28]:
df_bat = pd.read_csv('./ICC_rankings/ICCtestbattingrankings.csv')
df_bowl = pd.read_csv('./ICC_rankings/ICCtestbowlingrankings.csv')

In [29]:
for df_i in [df_bat, df_bowl]:
    df_i[['Initial','Surname']]=df_i['Name'].str.rsplit(n=1,expand=True)
    df_i['Surname'].fillna(df_i['Name'],inplace=True)
    df_i['Date'] = pd.to_datetime(df_i['Date'])

For each player, get their rating at start of each test match. Give a default rating of 200 batting and 50 bowling to those not in the top 100

#### These default ratings are fairly arbitrary at the moment. May want to amend this... Also this could be more efficient

In [30]:
for ap in ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11',\
        'p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22']:
    df_new[ap+'_bat_rating'] = 200 #### give default rating to those not in top 100  
    df_new[ap+'_bowl_rating'] = 50 #### give default rating to those not in top 100 
    
for df_i in [df_bat, df_bowl]:
    if df_i.equals(df_bat):
        discipline='_bat'
    elif df_i.equals(df_bowl):
        discipline='_bowl'
    else:
        print('something has gone wrong?')
        break
        
    counter=0
    for matchid in df_new['Match_id']:
        date = df_new[df_new['Match_id']==matchid]['Start_date'].values[0]
        counter+=1
        print('##############################',counter, date)
        top_players = df_i[df_i['Date']==date]['Surname'].values.tolist()
        for ap in ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11',\
            'p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22']:
            for player in df_new.loc[df_new['Match_id']==matchid, ap]:
                if isinstance(player, str): ### not nan (float)    
                    #print(player,player.split()[-1])
                    if  player.split()[-1] in top_players:
                        #print(f'{player} IS in top 100 {discipline} on {date}')
                        df_new.loc[df_new['Match_id']==matchid, ap+discipline+'_rating'] = df_i.loc[(df_i['Date']==date) & (df_i['Surname']==player.split()[-1]),'Rating'].values[0] 
                    else:
                        #print(f'{player} not in top 100 {discipline} on {date}')
                        pass

############################## 1 2011/08/04
############################## 2 2007/11/16
############################## 3 2016/05/19
############################## 4 2019/02/21
############################## 5 2010/08/03
############################## 6 2009/03/06
############################## 7 2013/11/06
############################## 8 2018/12/06
############################## 9 2010/12/26
############################## 10 2005/01/02
############################## 11 2020/12/26
############################## 12 2005/03/11
############################## 13 2012/11/25
############################## 14 2016/02/12
############################## 15 2012/11/17
############################## 16 2008/10/09
############################## 17 2019/08/14
############################## 18 2010/12/16
############################## 19 2007/05/18
############################## 20 2009/07/17
############################## 21 2016/07/28
############################## 22 2019/02/13
###################

############################## 182 2014/06/08
############################## 183 2006/12/26
############################## 184 2017/07/14
############################## 185 2009/12/04
############################## 186 2017/09/04
############################## 187 2017/12/14
############################## 188 2008/04/11
############################## 189 2016/11/17
############################## 190 2009/04/03
############################## 191 2007/12/08
############################## 192 2015/10/14
############################## 193 2014/08/07
############################## 194 2012/11/15
############################## 195 2011/12/26
############################## 196 2017/01/12
############################## 197 2014/07/17
############################## 198 2016/11/17
############################## 199 2020/08/05
############################## 200 2006/08/04
############################## 201 2016/12/08
############################## 202 2017/01/20
############################## 203

############################## 361 2013/02/14
############################## 362 2019/07/24
############################## 363 2006/11/11
############################## 364 2006/12/15
############################## 365 2006/03/16
############################## 366 2011/12/09
############################## 367 2018/11/30
############################## 368 2018/11/06
############################## 369 2012/11/13
############################## 370 2010/11/20
############################## 371 2016/11/09
############################## 372 2010/06/10
############################## 373 2010/10/09
############################## 374 2019/01/24
############################## 375 2011/11/14
############################## 376 2007/07/03
############################## 377 2012/12/05
############################## 378 2014/08/15
############################## 379 2011/11/09
############################## 380 2019/02/28
############################## 381 2007/11/22
############################## 382

############################## 15 2012/11/17
############################## 16 2008/10/09
############################## 17 2019/08/14
############################## 18 2010/12/16
############################## 19 2007/05/18
############################## 20 2009/07/17
############################## 21 2016/07/28
############################## 22 2019/02/13
############################## 23 2016/07/21
############################## 24 2013/03/22
############################## 25 2014/10/25
############################## 26 2015/11/05
############################## 27 2005/09/13
############################## 28 2009/03/19
############################## 29 2012/11/30
############################## 30 2014/11/03
############################## 31 2011/10/21
############################## 32 2016/08/06
############################## 33 2010/03/20
############################## 34 2010/03/19
############################## 35 2013/11/21
############################## 36 2008/12/11
##########

############################## 195 2011/12/26
############################## 196 2017/01/12
############################## 197 2014/07/17
############################## 198 2016/11/17
############################## 199 2020/08/05
############################## 200 2006/08/04
############################## 201 2016/12/08
############################## 202 2017/01/20
############################## 203 2015/06/11
############################## 204 2018/08/01
############################## 205 2008/07/18
############################## 206 2013/07/18
############################## 207 2006/06/22
############################## 208 2007/06/15
############################## 209 2008/07/31
############################## 210 2018/10/04
############################## 211 2006/12/07
############################## 212 2015/06/17
############################## 213 2006/03/09
############################## 214 2018/06/14
############################## 215 2009/12/16
############################## 216

############################## 374 2019/01/24
############################## 375 2011/11/14
############################## 376 2007/07/03
############################## 377 2012/12/05
############################## 378 2014/08/15
############################## 379 2011/11/09
############################## 380 2019/02/28
############################## 381 2007/11/22
############################## 382 2010/11/04
############################## 383 2018/01/04
############################## 384 2018/11/23
############################## 385 2012/03/07
############################## 386 2006/03/31
############################## 387 2006/04/03
############################## 388 2007/01/02
############################## 389 2007/07/19
############################## 390 2018/12/26
############################## 391 2014/06/16
############################## 392 2011/08/31
############################## 393 2013/04/25
############################## 394 2015/01/03
############################## 395

In [31]:
df_new.to_csv('test_match_summary_with_player_rankings.csv',index=False)